In [ ]:
import numpy as np
import pandas as pd
import time
import random
import datetime
from datetime import datetime, date, timedelta
from scipy import stats

parse_dates = ['time']
df=pd.read_csv ('finalDFPULITI.csv' , header=0,engine="python", quoting=0, parse_dates=parse_dates)
df.head()

In [ ]:
perc = 0.5  # it can be 0,1/0,2/0,3/0,4/0,5
errors = 5000*perc

In [ ]:
istanze = 5000
for i in range(istanze):
    print(i)
    coin = np.random.randint(0, 3)  # To decide whether an error is inserted, a virtual 'coin' is flipped (a random number between 0,1,2 is taken). If the result is 1, the ACCURACY error is injected, 2 COMPLETENESS error, 0 no error
    if(coin == 1 and errors > 0): 
        n = np.random.randint(25, 55) #ACCURACY error for fuelUsed
        errors = errors - 1
        if(i%2 == 1):
            n = n*(-1)
        df.loc[(df.case == i) & (df.event == 'manageinstallation_complete') , 'fuelUsed'] = n
    if(coin == 2 and errors > 0): #COMPLETENESS error for fuelUsed
        df.loc[(df.case == i) & (df.event == 'manageinstallation_complete') , 'fuelUsed'] = np.NaN
        errors = errors - 1
    

    
    coin = random.getrandbits(2)    # To decide whether an error is inserted, a virtual 'coin' is flipped (a random number between 0,1,2,3 is taken). If the result is 1, the ACCURACY error is injected, 2 CONSISTENCY error, 3 COMPLETENESS 0 no error
    if(coin == 1 and errors > 0):  
        wrongstatus = "open,inful,completed,finish, inf ul" # ACCURACY error for status in 'receive installation confirm' event
        n = np.random.randint(0,4)
        st = wrongstatus.split(",")[n]
        df.loc[(df.case == i) & (df.event == 'SA_SM_event_MESSAGE_receive installation confirm') , 'status'] = st
        errors = errors - 1
    
    
    if(coin == 2 and errors > 0):   # CONSISTENCY error for status in 'receive installation confirm' event
        wrongstatus = "open,active,finish,activated,attivated"
        n = np.random.randint(0,4)
        st = wrongstatus.split(",")[n]
        df.loc[(df.case == i) & (df.event == 'SA_SM_event_MESSAGE_receive installation confirm') , 'status'] = st
        errors = errors - 1

    if(coin == 3 and errors > 0):   # COMPLETENESS error for status in 'receive installation confirm' event
        df.loc[(df.case == i) & (df.event == 'SA_SM_event_MESSAGE_receive installation confirm') , 'status'] = np.NaN
        errors = errors - 1


    
    coin = random.getrandbits(2)
    if(coin == 1 and errors > 0):   # ACCURACY error for status in 'activate the service' event
        wrongstatus = "open,active,actv,finish,attivated"
        n = np.random.randint(0,4)
        st = wrongstatus.split(",")[n]
        df.loc[(df.case == i) & (df.event == 'activate the service') , 'status'] = st
        errors = errors - 1


        
    if(coin == 2 and errors > 0):   # CONSISTENCY error for status in 'activate the service' event
        wrongstatus = "open,active,in full,finish,attivated"
        n = np.random.randint(0,4)
        st = wrongstatus.split(",")[n]
        df.loc[(df.case == i) & (df.event == 'activate the service') , 'status'] = st
        errors = errors - 1   
    
    if(coin == 3 and errors > 0):   # COMPLETENESS error for status in 'activate the service' event
        df.loc[(df.case == i) & (df.event == 'activate the service') , 'status'] = np.NaN
        errors = errors - 1
    
    coin = np.random.randint(0, 3)
    pt = np.random.randint(-50,1)
    if(coin == 1 and errors > 0):   # ACCURACY error for paymentTerms
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_send invoice') , 'paymentTerms'] = pt
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_send solicitation with new amount') , 'paymentTerms'] = pt
        errors = errors - 1
    
    if(coin == 2 and errors > 0):   # COMPLETENESS error for paymentTerms
        errors = errors - 1
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_send invoice') , 'paymentTerms'] = np.NaN
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_send solicitation with new amount') , 'paymentTerms'] = np.NaN
    
    segno = 0
    coin = random.getrandbits(1)
    if(coin == 1 and errors > 0):   # COMPLETENESS error for t(CC_receive_order_request)
        errors = errors - 1
        segno = 1
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_CC_receive order request') , 'time'] = np.NaN

    coin = np.random.randint(0, 3)
    if(coin == 1 and errors > 0 and segno == 0):    # CONSISTENCY error for t(paymentReceipt)
        errors = errors - 1
        n = np.random.randint(45,250)
        ts = df.loc[(df.case == i) & (df.event == 'event_MESSAGE_CC_receive order request') , 'time']
        pd.to_datetime(ts)
        a = ts - timedelta(minutes = n)
        a = a.tolist()[0]
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_payment receipt') , 'time'] = a
    
    if(coin == 2 and errors > 0):   # COMPLETENESS error for t(paymentReceipt)
        df.loc[(df.case == i) & (df.event == 'event_MESSAGE_payment receipt') , 'time'] = np.NaN
        errors = errors - 1

    coin = np.random.randint(0, 3)
    if(coin == 1 and errors > 0 and segno == 0):    # CONSISTENCY error for t(sendInstallationConfirm)
        errors = errors - 1
        n = np.random.randint(45,250)
        ts = df.loc[(df.case == i) & (df.event == 'event_MESSAGE_CC_receive order request') , 'time']
        pd.to_datetime(ts)
        a = ts - timedelta(minutes = n)
        a = a.tolist()[0]
        df.loc[(df.case == i) & (df.event == 'message_sendInstallationConfirmToSA') , 'time'] = a
   
    if(coin == 2 and errors > 0):   # COMPLETENESS error for t(sendInstallationConfirm)
        errors = errors - 1
        df.loc[(df.case == i) & (df.event == 'message_sendInstallationConfirmToSA') , 'time'] = np.NaN 

In [ ]:
df.set_index("case", inplace = True)
df.to_csv('all' + str(perc) + 'simulationNumber.csv')